In [ ]:
import cv2
vidcap = cv2.VideoCapture('GX020203/GX020203.MP4')
success,image = vidcap.read()
count = 0
while success:
    saveFile = ('GX020203_pic/frame'+str(count)+'.jpg')
    #print(saveFile)
    img_write = cv2.imencode(".jpg", image)[1].tofile(saveFile)     # save frame as JPEG file      
    success,image = vidcap.read()
    print('Read a new frame: ', success)
    count += 1

In [125]:
import math

latitude = list()
longitude = list()
with open('GX020203/06_aligned_tomtom.txt') as f:
    for i, line in enumerate(f):
        
        lines = line.split(" ")
        latitude.append(float(lines[1]))
        longitude.append(float(lines[2]))
        
        
        
dir1 = list()
with open('GX020203/06_vbox_heading .txt') as f:
    for i, line in enumerate(f):
        
        lines = line.split(" ")
        dir1.append(float(lines[1]))

        

        
#print(lines[1])       
print(latitude[0],longitude[0])
print(len(latitude))
video_len = i+1
        

25.0242578 121.5529053
16036


In [126]:
import osmnx as ox
import pandas as pd

start_pos = (latitude[0] ,longitude[0])

tags = {'shop': True,'amenity':True}

G = ox.geometries.geometries_from_point(start_pos, tags, dist=2000)

/home/henry/anaconda3/lib/python3.9/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):
/home/henry/anaconda3/lib/python3.9/site-packages/osmnx/geometries.py:805: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_outer_linestring in list(merged_outer_linestrings):


In [127]:
def get_shop(x , y):
    df = pd.DataFrame()
    a=0
    eat=0
    num = list()

    if ("shop" or "amenity") in G:
        for idx in range(len(G)):
            if G['amenity'][idx] == "restaurant":
                pt = G['geometry'][idx].centroid
                if 10**5*(ox.distance.euclidean_dist_vec(x, y,pt.y,pt.x)) < 150:
                    B = ox.bearing.calculate_bearing( x, y, pt.y, pt.x)
                    L = ox.distance.euclidean_dist_vec(x, y, pt.y, pt.x)
                    df = pd.concat([df, pd.DataFrame.from_records([{'loc': pt ,"bearing": B,"length": L*10**5, "type":"restaurant","name": G['name'][idx] }])])
                    a +=1
                    eat +=1
            elif G['amenity'][idx] == "cafe":
                pt = G['geometry'][idx].centroid
                if 10**5*(ox.distance.euclidean_dist_vec(x, y,pt.y,pt.x)) < 150:
                    B = ox.bearing.calculate_bearing( x, y, pt.y, pt.x)
                    L = ox.distance.euclidean_dist_vec(x, y, pt.y, pt.x)
                    df = pd.concat([df, pd.DataFrame.from_records([{'loc': pt,"bearing": B,"length": L*10**5, "type":"cafe","name": G['name'][idx] }])])
                    a=a+1
            elif G['shop'][idx] == "convenience":
                pt = G['geometry'][idx].centroid
                if 10**5*(ox.distance.euclidean_dist_vec(x, y,pt.y,pt.x)) < 150:
                    B = ox.bearing.calculate_bearing( x, y, G['geometry'][idx].y, G['geometry'][idx].x)
                    L = ox.distance.euclidean_dist_vec(x, y, G['geometry'][idx].y, G['geometry'][idx].x)
                    df = pd.concat([df, pd.DataFrame.from_records([{'loc': G['geometry'][idx],"bearing": B,"length": L*10**5,"type":"CVS" }])])
                    a=a+1
    
    for i in range(a):
        if i < (a-eat):
            num.append("Nan")
        else:
            num.append(str(i-a+eat))
    #print(num)
     
    df = df.sort_values(by=['type','length'],ascending=True)
    df['Num'] = num
    df = df.reset_index()

    
   

    

                   
    if eat > 5:
        #df = df.sort_values(by=['type','length'],ascending=True)
        eat_num = a-eat + 5
        #df1 = df.drop([0, 3])
        df1 = df.drop(df.index[eat_num:])
    else:
        df1 = df
        

  

    return(df1)



    
get_shop(25.02764252,121.5553348)

    
    
    
#df = df.sort_values(by=['length'],ascending=False)

,index,loc,bearing,length,type,name,Num
0,0,POINT (121.5565004 25.0281833),62.886015,128.493827,CVS,NaN,Nan
1,0,POINT (121.554979 25.0276978),279.729874,36.006877,cafe,三日月,Nan
2,0,POINT (121.5552502 25.0276104),247.265716,9.049229,restaurant,馬友友印度廚房,0
3,0,POINT (121.5555077 25.0275731),113.898574,18.631572,restaurant,明月湯包,1
4,0,POINT (121.5556045 25.0275702),106.485382,27.922799,restaurant,Take out,2
5,0,POINT (121.5557273 25.0276342),91.340054,39.258817,restaurant,李福記排骨滷味,3
6,0,POINT (121.5555894 25.027296),146.346408,42.999683,restaurant,辣霸子,4


In [139]:
def get_dir(a,b,c,d):
    
    dir = ox.bearing.calculate_bearing(a, b, c, d)
    
    
    return(dir)

In [140]:
def get_pos(a, dir): #get the point should be at the top or bottom and change it angle to 0~180
    pos = a-float(dir)+60
    up = 1
    if pos<0:
        up = 0
        pos = pos * (-1)
    elif pos >120:
        up = 0
        pos = 360 - pos
    else:
        up = up
        pos = pos 
        
    return(pos,up)

In [141]:
def get_center(pos,up,len,wide,dist): #get center of the circle
    if up and dist <= 40:
        center = (int(pos* len//120),int(wide//2))
    elif up and dist > 40:
        center = (int(pos* len//120),int(wide//5))
    else:
        center = (int(pos* len//240),int(wide - 100))
    
    return(center)

In [142]:
def cir_size(a,up):#get its radius
    if up:
        size = 110
        
        if a == 0:
            size = 0
        else:
            for i in range(a):
                size = size - (1/2)
    else:
        size = 50
        
    return size

In [143]:
def c_color(a):#get the color from its type
    color = (225,0,0)
    if a == "CVS":
        color = (0,0,225)
    elif a =="restaurant":
        color = (173,13,106)
    elif a == "cafe":
        color = (0,225,0)
    else:
        color == (128, 128, 0)
        
    return(color)

In [144]:
def get_txt_size(a):#get the word size
    size = 0.8
    for i in range(a-50):
        size = size + 0.017
            
    return size

In [145]:
import numpy as np
import cv2



  
def draw(df,dir,img,cnt):
    for idx in range(len(df)):

        bear = df.iat[idx,2]
        dist = int(df.iat[idx,3])
        typ = str(df.iat[idx,4])
        #print("bear:"+bear)
        #print("dist: "+dist)
        #print("typ: "+typ)



        pos,up = get_pos(bear,dir)


        CENTER = get_center(pos,up,img.shape[1],img.shape[0],dist)

        c_size = int(cir_size(dist,up))
    
        color = c_color(typ) 
    
    
    
        zero = np.zeros((img.shape),dtype = np.uint8)
        zero_mask = cv2.circle(zero, CENTER, c_size, color, -1)
        

        alpha = 1
        beta = 0.8
        gamma = 0
    
        img = cv2.addWeighted(img,alpha,zero_mask,beta,gamma)
    

    
    



        TEXT_FACE = cv2.FONT_HERSHEY_DUPLEX
        TEXT_SCALE = get_txt_size(c_size)
        TEXT_THICKNESS = 2
        TEXT1 = str(dist)
        TEXT2 = typ


        text_size, _ = cv2.getTextSize(TEXT1, TEXT_FACE, TEXT_SCALE, TEXT_THICKNESS)
        text_origin = (CENTER[0] - text_size[0] // 2, CENTER[1] + 4*text_size[1]//3 )

        cv2.putText(img, TEXT1, text_origin, TEXT_FACE, TEXT_SCALE, (0,0,0), TEXT_THICKNESS, cv2.LINE_AA)


        text_size, _ = cv2.getTextSize(TEXT2, TEXT_FACE, TEXT_SCALE, TEXT_THICKNESS)
        text_origin = (CENTER[0] - text_size[0] // 2, CENTER[1] - 4*text_size[1]//3 )

        cv2.putText(img, TEXT2, text_origin, TEXT_FACE, TEXT_SCALE, (0,0,0), TEXT_THICKNESS, cv2.LINE_AA)
    


    pt = (0,int(img.shape[0]-img.shape[0]//9))    
    word = "heading: " + str(dir)
    cv2.putText(img, word, pt, cv2.FONT_HERSHEY_SIMPLEX,1, (0, 255, 255), 2, cv2.LINE_AA)    
    
    #cv2.imwrite('pic'+str(cnt)+'.png',img)
    saveFile = ('GX020203_vid5/frame'+str(cnt)+'.jpg')
    img_write = cv2.imencode(".jpg", img)[1].tofile(saveFile)

    

In [146]:
def main():
    dir = list()
    for i in range(video_len):
        img = cv2.imread('GX020203_pic/frame'+str(i)+'.jpg')
        
        if i == 0:
            current_pos = (float(latitude[i]),float(longitude[i]))
            last_pos = (0,0)
        else:
            last_pos = (float(latitude[i-1]),float(longitude[i-1]))
            current_pos = (float(latitude[i]),float(longitude[i]))
            
 
        '''
    #設1公尺的話因為走一部通常不超過一公尺 所以會導致bearing錯誤
        if (10**5*(ox.distance.euclidean_dist_vec(current_pos[1], current_pos[0],last_pos[1],last_pos[0]))< 0.3):
            last_pos = (float(latitude[i-1]),float(longitude[i-1]))
            current_pos = last_pos
        else:
            last_pos = (float(latitude[i-1]),float(longitude[i-1]))
            current_pos = (float(latitude[i]),float(longitude[i]))
'''
            
        df = get_shop(current_pos[0],current_pos[1])
        
        #dir = list()
        
        dir_crt = get_dir(last_pos[0],last_pos[1],current_pos[0],current_pos[1])
        

        if last_pos == current_pos:
            dir.append(dir[i-1])
            #print(dir[i-1],i-1)
        else:
            dir.append(get_dir(last_pos[0],last_pos[1],current_pos[0],current_pos[1]))
            
            if abs(dir[i]-dir[i-1])>180:
                dir_crt = dir_crt - 180
            else:
                dir_crt = dir_crt
            
            
            #smoothing to the angle, if angle 
            if abs(dir[i] - dir[i-1])<25:
                dir[i] = dir[i-1]
            else:
                dir[i] = dir[i]
                
                
             #設1公尺的話因為走一部通常不超過一公尺 所以會導致bearing錯誤   
            if (10**5*(ox.distance.euclidean_dist_vec(current_pos[1], current_pos[0],last_pos[1],last_pos[0]))< 0.3):
                dir[i] = dir[i-1]
            else:
                dir[i] = dir[i]
            
        

            
            
        #print(dir[0])    
        
        draw(df,dir[i],img,i)
        print(str(i)+'/16036')

In [ ]:
main()

In [ ]:
import cv2
import os
import random

img = cv2.imread('GX020203_vid5/frame0.jpg')
size = (img.shape[1],img.shape[0])
print(size)
fourcc = cv2.VideoWriter_fourcc(*"XVID")
videoWrite = cv2.VideoWriter('new_vid5.avi',fourcc,30,size)


files = os.listdir('GX020203_vid5/')
out_num = len(files)
print(out_num)
for i in range(0,out_num):
    filename = 'GX020203_vid5/frame'+str(i)+'.jpg'
    img = cv2.imread(filename)
    videoWrite.write(img)
    print(str(i)+"/16036")
videoWrite.release